In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import os
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, InputLayer
from sklearn.model_selection import train_test_split

In [2]:
dataset_crop_weeds = 'agri_data/data/'  # Update this path as necessary

# Images Sizes
img_height = 512
img_width = 512

# Defines batch size and number of epochs
batch_size = 32
epochs = 10


In [3]:
images = []
annotations = []

for filename in os.listdir(dataset_crop_weeds):
    if filename.endswith('.jpeg'):
        image_path = os.path.join(dataset_crop_weeds, filename)
        images.append(image_path)
    elif filename.endswith('.txt'):
        annotation_path = os.path.join(dataset_crop_weeds, filename)
        annotations.append(annotation_path)

print(f"Total Images: {len(images)}")
print(f"Total Annotations: {len(annotations)}")


Total Images: 1300
Total Annotations: 1300


In [4]:
train_images, test_images, train_annotations, test_annotations = train_test_split(images, annotations, test_size=0.2, random_state=42)
train_images, val_images, train_annotations, val_annotations = train_test_split(train_images, train_annotations, test_size=0.1, random_state=42)


In [5]:
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    image = cv2.resize(image, (img_width, img_height))
    return image / 255.0  # Normalize pixel values to [0, 1]


In [6]:
def parse_annotation(annotation_path):
    with open(annotation_path, 'r') as file:
        lines = file.readlines()

    annotations = []
    for line in lines:
        class_label = int(line.strip().split()[0])  # Assuming class label is the first element
        annotations.append(class_label)  # For binary classification

    return annotations


In [9]:
# Assuming each annotation file contains a single integer label for binary classification
def parse_annotation(annotation_path):
    with open(annotation_path, 'r') as file:
        # Read the first line to get the class label (assuming the class label is always the first piece of data on the line)
        first_line = file.readline().strip()
        class_label = int(first_line.split()[0])  # Extracts the class label (integer) from the line
    return class_label

# Preprocess the image and annotation data
train_images_data = np.array([load_and_preprocess_image(img) for img in train_images])
val_images_data = np.array([load_and_preprocess_image(img) for img in val_images])
test_images_data = np.array([load_and_preprocess_image(img) for img in test_images])

# Preprocess the image and annotation data again with the corrected parse_annotation function
train_annotations_data = np.array([parse_annotation(ann) for ann in train_annotations])
val_annotations_data = np.array([parse_annotation(ann) for ann in val_annotations])
test_annotations_data = np.array([parse_annotation(ann) for ann in test_annotations])



In [10]:
model = tf.keras.Sequential([
    InputLayer(input_shape=(img_height, img_width, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [11]:
history = model.fit(train_images_data, train_annotations_data, batch_size=batch_size, epochs=epochs, validation_data=(val_images_data, val_annotations_data))


Epoch 1/10
30/30 [==============================] - 82s 3s/step - loss: 1.2570 - accuracy: 0.5096 - val_loss: 0.6934 - val_accuracy: 0.4712
Epoch 2/10
30/30 [==============================] - 96s 3s/step - loss: 0.6995 - accuracy: 0.5438 - val_loss: 0.6928 - val_accuracy: 0.5385
Epoch 3/10
30/30 [==============================] - 95s 3s/step - loss: 0.6913 - accuracy: 0.5470 - val_loss: 0.6930 - val_accuracy: 0.5192
Epoch 4/10
30/30 [==============================] - 85s 3s/step - loss: 0.6930 - accuracy: 0.4904 - val_loss: 0.6929 - val_accuracy: 0.5192
Epoch 5/10
30/30 [==============================] - 83s 3s/step - loss: 0.6917 - accuracy: 0.5267 - val_loss: 0.6922 - val_accuracy: 0.5288
Epoch 6/10
30/30 [==============================] - 84s 3s/step - loss: 0.6597 - accuracy: 0.6186 - val_loss: 0.7494 - val_accuracy: 0.5673
Epoch 7/10
30/30 [==============================] - 104s 3s/step - loss: 0.5334 - accuracy: 0.7778 - val_loss: 0.9647 - val_accuracy: 0.5096
Epoch 8/10
30/30 [=

In [12]:
test_loss, test_acc = model.evaluate(test_images_data, test_annotations_data, verbose=2)
print(f"Test accuracy: {test_acc}")


9/9 - 7s - loss: 1.3945 - accuracy: 0.4885 - 7s/epoch - 723ms/step
Test accuracy: 0.48846152424812317
